In [1]:
#Import libraries
from IPython.display import display
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# Define url for request, will update to loop through urls.
url = 'https://www.londonstockexchange.com/exchange/prices/stocks/summary/fundamentals.html?fourWayKey=IL0010943905GBGBXASX1'

In [3]:
# Request and parse url content
response = requests.get(url).content
soup = BeautifulSoup(response, 'html.parser')

In [4]:
# Get company name 
breadcrumb  = soup.find('ol', {'class' : 'breadcrumb'})
name = breadcrumb.find('li', {'class' : 'active'}).text.strip()

In [5]:
# Put years in list
years = []
Income = soup.find('th', {'class' : 'name'})
conts = Income.parent.find_all('th')    
for date in conts:
    final = re.findall("[0-9]{2}-[a-zA-z]{3}-[0-9]{2}", date.text.strip())
    if not len(final) == 0:
        years.append(final[0])

In [6]:
# Get currency
currency = []
for cur in conts:
    fstring = cur.text.strip()
    final = fstring.find('(')
    if final != -1:
        final = fstring[final + 2]
        currency.append(final)
    else:
        continue

In [7]:
# Get units
units = []
for unit in conts:
    fstring = str(unit.text.strip())
    final = fstring.find('(')
    if final != -1:
        final = fstring[final + 5]
        units.append(final)
    else:
        continue

In [8]:
# Get data from continuing operations table and store in dictionary
value = []
indicator = []
Continuing_Ops = soup.find("td", { 'class' : 'name writeRowSection'})

income_table = Continuing_Ops.parent.parent

n = 0
for tag in income_table.find_all('tr'):
    n = n + 1

start = Continuing_Ops.parent
for tag in income_table.find_all('tr'):
    #start = tag.child
    starts = tag.find_all('td')
    for numbers in starts:
        if numbers.find('td', {'class' : 'name writeRowSection'}):
            starts.next()
        elif numbers.text.strip() == 'n/a':
            value.append('n/a')
        elif bool(re.search(r'[a-zA-Z]', numbers.text.strip())):
            for i in range(len(starts) - 1):
                indicator.append(numbers.text.strip())
        if bool(re.search(r'\d', numbers.text.strip())):
            value.append(numbers.text.strip())

In [21]:
# Combine lists in dictionary

# Check if there is an error in the length of lists
if len(indicator) != len(value):
    display("Error dictionary items of unequal length")

# Prepare lists
years = years * int(len(indicator)/len(years))
units = units * int(len(indicator)/len(units))
currency = currency * int(len(indicator)/len(currency))

# Define keys
key_strings = ["years", "indicator", "value", "units", "currency"]
keys = [years, indicator, value, units, currency]
financial_data = {}
financial_data['Name'] = name
for i in range(len(key_strings)):
    financial_data[key_strings[i]] = []
    
# Recursively add data for each key
for i in range(5):
    for data in keys[i]:
        financial_data[key_strings[i]].append(data)    

In [22]:
# Create dataframe from dictionary object
df = pd.DataFrame(financial_data)
df

,Name,years,indicator,value,units,currency
0,AMIAD WATER,31-Dec-14,Revenue,118.47,m,$
1,AMIAD WATER,31-Dec-15,Revenue,118.78,m,$
2,AMIAD WATER,31-Dec-16,Revenue,105.59,m,$
3,AMIAD WATER,31-Dec-17,Revenue,112.27,m,$
4,AMIAD WATER,31-Dec-18,Revenue,113.92,m,$
...,...,...,...,...,...,...
75,AMIAD WATER,31-Dec-14,Dividend per Share,2.00¢,m,$
76,AMIAD WATER,31-Dec-15,Dividend per Share,6.70¢,m,$
77,AMIAD WATER,31-Dec-16,Dividend per Share,3.20¢,m,$
78,AMIAD WATER,31-Dec-17,Dividend per Share,3.20¢,m,$


In [23]:
# Export to csv for upload
export_csv = df.to_csv (r'C:\Users\Harry Shuttleworth\Desktop\Python\LSE_data.csv', index = None, header=True)